#### The overlate function can be applied to a VCF to translate genotypes to the data structure neccessary for [Hscan](https://messerlab.org/resources/)
To run this, make 2 empty folders, and put its path in the 'indir' and 'outdir' objects below

In [ ]:
indir <- "/home/jmiller1/soft_sweep/vcf/"
system(## do a system call to get a header to put into indir)
header <- scan(paste(indir,'vcf.header',sep=''),what='character')
outdir <- "/home/jmiller1/soft_sweep/output"

In [1]:
overlate <- function(X,Phased=FALSE){
    ref <- X[4]
    alt <- X[5]
    
    if(nchar(ref)>1|nchar(alt)>1){
        if(nchar(ref)>nchar(alt)){
            ref <- 'I'
            alt <- 'D'
        } else {
            ref <- 'D'
            alt <- 'I'
        }
    }
    
    if(Phased==TRUE){
        trans <- c(paste(ref,ref,sep='/'),
            paste(ref,alt,sep='/'),
            paste(alt,ref,sep='/'),
            paste(alt,alt,sep='/'),
            'N/N')
    } else {
    # Unphased
        trans <- c(ref,'.','.',alt,'N')
            names(trans) <- c('0/0','0/1','1/0','1/1','.')
    }              
    return(trans[gsub("\\:.*",'',X[10:length(X)])]) 
    
}

In [ ]:
out.Hscan <- function(scaf){

    system(paste("bgzip -cd /home/nreid/popgen/variants/bowfree/ALL1/hallsnps.vcf.gz | awk \'$1==\"Scaffold", scaf,"\"{print $0}\' > in.vcf",sep=''))    
    
    NBH <- read.table(paste(indir,'in.vcf',sep=''), stringsAsFactors=FALSE, header=TRUE) 
    colnames(NBH) <- header
        
    BI <- data1[,c(1:10,grep('BI',colnames(data1)))]
    NBH <- data1[,c(1:10,grep('NBH',colnames(data1)))]

    dat.NBH <- t(apply(NBH,1,overlate))
    colnames(dat.NBH) <- colnames(NBH[,10:ncol(data1)])
    write.table(data.frame(pos=data1$POS, dat),file=paste(outdir,scaf,sep='/'),quote=F,sep=",", col.names=FALSE, row.names=FALSE) 

    dat.BI <- t(apply(BI,1,overlate))
    colnames(dat.BI) <- colnames(BI[,10:ncol(BI)])
    write.table(data.frame(pos=BI$POS, dat.BI),file=paste(paste(outdir,scaf,sep='/'),'BI',sep=''),quote=F,sep=",", col.names=FALSE, row.names=FALSE) 
 
}



In [ ]:
get.plots <- function(pop,scaffold){
    
    hstat.NBH <- read.table(file=paste(paste(outdir,scaf,sep='/'),'NBH',sep=''), header=T)
    hstat.BI <- read.table(file=paste(paste(outdir,scaf,sep='/'),'BI',sep=''), header=T)

    ylim.max <- max(c(hstat.NBH$H,hstat.BI$H))
    
    if(pop=='NBH'){
        plot(hstat.NBH$x,hstat.NBH$H/1000,
            pch=20,ylim=c(0,ylim.max))
    } else {
        plot(hstat.BI$x,hstat.BI$H/1000,
            pch=20,ylim=c(0,ylim.max))
    }
    
}

Reads in the scaf and headers the dataframe with sample ids

In [ ]:
scaf <- c(0,217)

In [ ]:
sapply(scaf,out.Hscan)

get.plots(pop='NBH',scaffold=0)